In [109]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools
from sklearn.linear_model import PassiveAggressiveClassifier

### Data Exploration

In [110]:
import pandas as pd

# Import `fake_or_real_news.csv` 
file = pd.read_csv("fake_or_real_news.csv")
    
# Inspect shape of `df` 
file.shape

# Print first lines of `df` 
file.head()

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,title,text,label,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
#Setting the DataFrame index (row labels) using one or more existing columns
# Set index 
file = file.set_index("Unnamed: 0")

# Print first lines of `df` 
file.head()

,title,text,label,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I want to separate the labels and set up training and test datasets.
For this notebook, I decided to focus on using the longer article text.
Because I knew I would be using bag-of-words and Term Frequency–Inverse Document Frequency (TF-IDF) to extract features, 
this seemed like a good choice.
Using longer text will hopefully allow for distinct words and features for my real and fake news data.

In [112]:
# Set `y` 
y = df.label 
y.head()

Unnamed: 0
8476     FAKE
10294    FAKE
3608     REAL
10142    FAKE
875      REAL
Name: label, dtype: object

In [113]:
# Drop the `label` column
file.drop("label", axis=1)
file.head()

,title,text,label,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
# Make training and test sets 
X_train, X_test, y_train, y_test = train_test_split(file['text'].values.astype('str'), y , test_size=0.33, random_state=53)

### Building Vectorizer Classifiers

 Now that you have your training and testing data, you can build your classifiers. 
To get a good idea if the words and tokens in the articles had a significant impact on whether the news was fake or real,
you begin by using CountVectorizer and TfidfVectorizer.

You’ll see the example has a max threshhold set at .7 for the TF-IDF vectorizer tfidf_vectorizer using the max_df argument.
This removes words which appear in more than 70% of the articles.
Also, the built-in stop_words parameter will remove English stop words from the data before making vectors.

### 1. By Using Count Vectorizer

In [143]:
# Initialize the `count_vectorizer` 
count_vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the training data 
count_train = count_vectorizer.fit_transform(X_train) 

# Transform the test set 
count_test = count_vectorizer.transform(X_test)

print(count_vectorizer.get_feature_names())

print('\n')

print(count_train.toarray())

['00', '000', '000000031', '00006', '0001pt', '0002', '000billion', '000ft', '000km', '001', '003', '004', '004s', '005', '005s', '006', '00684', '007', '007s', '008s', '009', '00am', '00pm', '01', '010', '011', '013', '013c2812c9', '016', '018', '01am', '02', '020', '022', '024', '027', '02714', '028', '02870', '03', '030', '031', '032', '0325', '033', '035', '03747', '039', '04', '040', '0400', '042', '044', '047', '049', '04pm', '05', '050', '052', '053', '056', '058', '06', '0600', '062', '063', '0640', '066', '068', '06pm', '07', '071', '075', '0750', '079', '08', '081', '084', '0843', '085', '0851', '0891', '09', '091', '09am', '09pm', '0_65b67362bd', '0_jgdktlmn', '0_kvyhphja', '0b6njlny5j', '0in', '0jsn6pjkan', '0oeekvljlt', '0pt', '0womdwalmi', '10', '100', '1000', '1000s', '10021', '10028', '100515p', '100bn', '100c', '100k', '100m', '100percentfedup', '100s', '100th', '101', '1010359', '1019', '101st', '102', '1026', '102816', '103', '1033', '104', '1040', '104893', '104th',

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### 2. By Using Tf-IDF Vectorizer

In [144]:
# Initialize the `tfidf_vectorizer` 
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7) 

# Fit and transform the training data 
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 

# Transform the test set 
tfidf_test = tfidf_vectorizer.transform(X_test)

print(tfidf_vectorizer.get_feature_names())

print(tfidf_train.shape) 
# The result will show that there are 5238 documents and 56338 distinct features(words): (5238,56338)

['00', '000', '000000031', '00006', '0001pt', '0002', '000billion', '000ft', '000km', '001', '003', '004', '004s', '005', '005s', '006', '00684', '007', '007s', '008s', '009', '00am', '00pm', '01', '010', '011', '013', '013c2812c9', '016', '018', '01am', '02', '020', '022', '024', '027', '02714', '028', '02870', '03', '030', '031', '032', '0325', '033', '035', '03747', '039', '04', '040', '0400', '042', '044', '047', '049', '04pm', '05', '050', '052', '053', '056', '058', '06', '0600', '062', '063', '0640', '066', '068', '06pm', '07', '071', '075', '0750', '079', '08', '081', '084', '0843', '085', '0851', '0891', '09', '091', '09am', '09pm', '0_65b67362bd', '0_jgdktlmn', '0_kvyhphja', '0b6njlny5j', '0in', '0jsn6pjkan', '0oeekvljlt', '0pt', '0womdwalmi', '10', '100', '1000', '1000s', '10021', '10028', '100515p', '100bn', '100c', '100k', '100m', '100percentfedup', '100s', '100th', '101', '1010359', '1019', '101st', '102', '1026', '102816', '103', '1033', '104', '1040', '104893', '104th',

In [145]:
print(tfidf_test)

  (0, 55519)	0.014912196932522385
  (0, 55510)	0.015039401224956606
  (0, 55195)	0.02759719599299367
  (0, 55156)	0.0165656914718902
  (0, 55060)	0.01936356976449145
  (0, 54706)	0.05494135533293899
  (0, 54623)	0.01716764942731099
  (0, 54383)	0.022849287415610185
  (0, 54365)	0.04497543426141779
  (0, 54177)	0.01753850374887669
  (0, 54157)	0.0288670436876859
  (0, 54096)	0.019096675279396166
  (0, 53975)	0.030000753050703696
  (0, 53970)	0.058382750938431385
  (0, 53890)	0.10233918671068942
  (0, 53831)	0.01923668678687172
  (0, 53826)	0.05360253254896542
  (0, 53825)	0.01888436883952796
  (0, 53758)	0.05885564782473221
  (0, 53669)	0.029637142896367503
  (0, 53522)	0.02376969483303334
  (0, 53465)	0.025538057250293802
  (0, 53375)	0.029853004147357098
  (0, 53027)	0.04949548203504776
  (0, 52674)	0.053682060730989384
  :	:
  (2579, 2741)	0.05576752869347099
  (2579, 2739)	0.05432073111418792
  (2579, 2736)	0.11320693798224199
  (2579, 2714)	0.021670722960678946
  (2579, 2680)	0.050

There are clearly comments, measurements or other nonsensical words as well as multilingual articles in the dataset that
you have been using. Normally, you would want to spend more time preprocessing this and removing noise, but as this tutorial 
just showcases a small proof of concept, you will see if the model can overcome the noise and properly
classify despite these issues.

In [146]:
# Get the feature names of `tfidf_vectorizer` 
print(tfidf_vectorizer.get_feature_names()[-10:])

# Get the feature names of `count_vectorizer` 
print(count_vectorizer.get_feature_names()[:10])

['תאמצנה', 'תוצאה', 'תחל', 'תיירות', 'תנותק', 'תעודת', 'תתרכז', 'القادمون', 'عربي', 'ยงade']
['00', '000', '000000031', '00006', '0001pt', '0002', '000billion', '000ft', '000km', '001']


### Count versus TF-IDF Features: Values of Token Generated by Both The Vectorizers Are Same

I was curious if my count and TF-IDF vectorizers had extracted different tokens. To take a look and compare features, you can extract the vector information back into a DataFrame to use easy Python comparisons.

As you can see by running the cells below, both vectorizers extracted the same tokens, but obviously have different weights. Likely, changing the max_df and min_df of the TF-IDF vectorizer could alter the result and lead to different features in each

In [147]:
count_df = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())
difference = set(count_df.columns) - set(tfidf_df.columns)
difference

set()

In [148]:
count_df.head()

,00,000,000000031,00006,0001pt,0002,000billion,000ft,000km,001,...,תאמצנה,תוצאה,תחל,תיירות,תנותק,תעודת,תתרכז,القادمون,عربي,ยงade
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
tfidf_df.head()

,00,000,000000031,00006,0001pt,0002,000billion,000ft,000km,001,...,תאמצנה,תוצאה,תחל,תיירות,תנותק,תעודת,תתרכז,القادمون,عربي,ยงade
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.180133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Comparing Models

Now it's time to train and test your models.

Here, you'll begin with an NLP favorite, MultinomialNB. You can use this to compare TF-IDF versus bag-of-words. My intuition was that bag-of-words (aka CountVectorizer) would perform better with this model. (For more reading on multinomial distribution and why it works best with integers, check out this fairly succinct explanation from a UPenn statistics course).

I personally find Confusion Matrices easier to compare and read, so I used the scikit-learn documentation to build some easily-readable confusion matrices (thanks open source!). A confusion matrix shows the proper labels on the main diagonal (top left to bottom right). The other cells show the incorrect labels, often referred to as false positives or false negatives. Depending on your problem, one of these might be more significant. For example, for the fake news problem, is it more important that we don't label real news articles as fake news? If so, we might want to eventually weight our accuracy score to better reflect this concern.

Other than Confusion Matrices, scikit-learn comes with many ways to visualize and compare your models. One popular way is to use a ROC Curve. There are many other ways to evaluate your model available in the scikit-learn metrics module.

### Confusion Matrix

In [150]:
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [153]:
clf = MultinomialNB() 
clf.fit(tfidf_train, y_train)
pred = clf.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred, labels=['FAKE', 'REAL'])
plot_confusion_matrix(cm, classes=['FAKE', 'REAL'])

ValueError: Unknown label type: (array([nan, nan, 'FAKE', ..., 'FAKE',
       ' and I am prepared to say it again. Yugoslavia', 'FAKE'],
      dtype=object),)